In [32]:
# for demo overrides write-host
if ((test-path .\utilities.ps1)) {. .\utilities.ps1 -useArm}

In [33]:
$global:clusterName = '{{resourceGroupName}}'
$global:resourceGroupName = '{{resourceGroupName}}'

$error.clear()
if(!(get-command get-azcontext)) {
    set-executionPolicy -executionPolicy Bypass
    install-module az.accounts
}

if(!(get-azcontext)) {
    $error.clear()
    import-module az.accounts
    connect-azaccount
}

if($error) {
   $error
}
else {
    'connected'
}

connected

In [3]:
# dont use as json. use get-azresource for json
$cluster = get-azservicefabriccluster -Name $clusterName -ResourceGroupName $resourceGroupName
write-host ($cluster)

PSClusterString : AddOnFeatures : 
 AvailableClusterVersions :
 CodeVersion : 7.0.466.9590
 SupportExpiryUtc : 12/31/9999 23:59:59
 Environment : Windows
 AzureActiveDirectory : 
 Certificate :
 Thumbprint : xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
 ThumbprintSecondary : 
 X509StoreName : My
 CertificateCommonNames : 
 ClientCertificateCommonNames :
 ClientCertificateThumbprints :
 ClusterCodeVersion : 7.0.466.9590
 ClusterEndpoint : https://{{location}}.servicefabric.azure.com/runtime/clusters/xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
 ClusterId : xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
 ClusterState : Ready
 DiagnosticsStorageAccountConfig :
 StorageAccountName : {{Microsoft.Storage/storageAccounts}}
 ProtectedAccountKeyName : StorageAccountKey1
 BlobEndpoint : https://{{Microsoft.Storage/storageAccounts}}.blob.core.windows.net/
 QueueEndpoint : https://{{Microsoft.Storage/storageAccounts}}.queue.core.windows.net/
 TableEndpoint : https://{{Microsoft.Storage/storageAccounts}}.table.core.windows.net/
 EventStoreServiceEnabled : 
 FabricSettings :
 Name : Security
 Parameters :
 Name : ClusterProtectionLevel
 Value : EncryptAndSign
 Name : Management
 Parameters :
 Name : CleanupApplicationPackageOnProvisionSuccess
 Value : True
 Name : CleanupUnusedApplicationTypes
 Value : True
 Name : PeriodicCleanupUnusedApplicationTypes
 Value : True
 Name : TriggerAppTypeCleanupOnProvisionSuccess
 Value : True
 Name : MaxUnusedAppTypeVersionsToKeep
 Value : 3
 ManagementEndpoint : https://{{resourceGroupName}}.{{location}}.cloudapp.azure.com:19080
 NodeTypes :
 NodeTypeDescription :
 Name : {{Microsoft.Compute/virtualMachineScaleSets}}
 PlacementProperties : 
 Capacities : 
 ClientConnectionEndpointPort : 19000
 HttpGatewayEndpointPort : 19080
 DurabilityLevel : Bronze
 ApplicationPorts :
 StartPort : 20000
 EndPort : 30000
 EphemeralPorts :
 StartPort : 49152
 EndPort : 65534
 IsPrimary : True
 VmInstanceCount : 3
 ReverseProxyEndpointPort : 19081
 ProvisioningState : Succeeded
 ReliabilityLevel : Bronze
 ReverseProxyCertificate : 
 ReverseProxyCertificateCommonNames : 
 UpgradeDescription : 
 UpgradeMode : Automatic
 VmImage : Windows
 Id : /subscriptions/{{subscription}}/resourcegroups/{{resourceGroupName}}/providers/Microsoft.ServiceFabric/clusters/{{resourceGroupName}}
 Name : {{resourceGroupName}}
 Type : Microsoft.ServiceFabric/clusters
 Location : {{location}}
 Tags :
 Etag : 
 
AddOnFeatures : 
AvailableClusterVersions : {Microsoft.Azure.Management.ServiceFabric.Models.ClusterVersionDetails}
AzureActiveDirectory : 
Certificate : Microsoft.Azure.Management.ServiceFabric.Models.CertificateDescription
CertificateCommonNames : 
ClientCertificateCommonNames : {}
ClientCertificateThumbprints : {}
ClusterCodeVersion : 7.0.466.9590
ClusterEndpoint : https://{{location}}.servicefabric.azure.com/runtime/clusters/xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
ClusterId : xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
ClusterState : Ready
DiagnosticsStorageAccountConfig : Microsoft.Azure.Management.ServiceFabric.Models.DiagnosticsStorageAccountConfig
EventStoreServiceEnabled : 
FabricSettings : {Security, Management}
ManagementEndpoint : https://{{resourceGroupName}}.{{location}}.cloudapp.azure.com:19080
NodeTypes : {{{Microsoft.Compute/virtualMachineScaleSets}}}
ProvisioningState : Succeeded
ReliabilityLevel : Bronze
ReverseProxyCertificate : 
ReverseProxyCertificateCommonNames : 
UpgradeDescription : 
UpgradeMode : Automatic
VmImage : Windows
Id : /subscriptions/{{subscription}}/resourcegroups/{{resourceGroupName}}/providers/Microsoft.ServiceFabric/clusters/{{resourceGroupName}}
Name : {{resourceGroupName}}
Type : Microsoft.ServiceFabric/clusters
Location : {{location}}
Tags : {}
Etag :

In [31]:
# convert sf cluster az resource to sf arm template
$cluster = get-azresource -Name $clusterName -ResourceGroupName $resourceGroupName -ResourceType Microsoft.ServiceFabric/clusters
$clusterPropertiesJson = $cluster.properties | convertto-json -depth 99

$clusterResourcesObject = @{}
$clusterResourcesObject.'$schema' = "http://schema.management.azure.com/schemas/2015-01-01/deploymentTemplate.json"
$clusterResourcesObject.contentVersion = "xxx.xxx.xxx.xxx"

$clusterObject = @{}
$clusterObject.apiVersion = '2019-06-01-preview'
#$clusterObject.dependsOn = @()
$clusterObject.type = $cluster.Type
$clusterObject.location = $cluster.Location
#$clusterObject.id = $cluster.Id
$clusterObject.name = $cluster.Name
$clusterObject.tags = $cluster.Tags
#$clusterObject.etag = $cluster.ETag
$clusterObject.properties = $clusterPropertiesJson | convertfrom-json

$clusterResourcesObject.Add('resources',@($clusterObject))

$clusterTemplate = $clusterResourcesObject | convertto-json -depth 99
$clusterTemplate | out-file .\clustertemplate.json

#$clusterTemplate
write-host $clusterTemplate

{
 "contentVersion": "xxx.xxx.xxx.xxx",
 "resources": [
 {
 "name": "{{resourceGroupName}}",
 "apiVersion": "2019-06-01-preview",
 "properties": {
 "provisioningState": "Succeeded",
 "clusterId": "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx",
 "clusterCodeVersion": "7.0.466.9590",
 "clusterState": "Ready",
 "managementEndpoint": "https://{{resourceGroupName}}.{{location}}.cloudapp.azure.com:19080",
 "clusterEndpoint": "https://{{location}}.servicefabric.azure.com/runtime/clusters/xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx",
 "certificate": {
 "thumbprint": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
 "x509StoreName": "My"
 },
 "clientCertificateThumbprints": [],
 "clientCertificateCommonNames": [],
 "fabricSettings": [
 {
 "name": "Security",
 "parameters": [
 {
 "name": "ClusterProtectionLevel",
 "value": "EncryptAndSign"
 }
 ]
 },
 {
 "name": "Management",
 "parameters": [
 {
 "name": "CleanupApplicationPackageOnProvisionSuccess",
 "value": "True"
 },
 {
 "name": "CleanupUnusedApplicationTypes",
 "value": "True"
 },
 {
 "name": "PeriodicCleanupUnusedApplicationTypes",
 "value": "True"
 },
 {
 "name": "TriggerAppTypeCleanupOnProvisionSuccess",
 "value": "True"
 },
 {
 "name": "MaxUnusedAppTypeVersionsToKeep",
 "value": "3"
 }
 ]
 }
 ],
 "vmImage": "Windows",
 "reliabilityLevel": "Bronze",
 "nodeTypes": [
 {
 "name": "{{Microsoft.Compute/virtualMachineScaleSets}}",
 "vmInstanceCount": 3,
 "clientConnectionEndpointPort": 19000,
 "httpGatewayEndpointPort": 19080,
 "applicationPorts": {
 "startPort": 20000,
 "endPort": 30000
 },
 "ephemeralPorts": {
 "startPort": 49152,
 "endPort": 65534
 },
 "isPrimary": true,
 "reverseProxyEndpointPort": 19081,
 "durabilityLevel": "Bronze"
 }
 ],
 "diagnosticsStorageAccountConfig": {
 "storageAccountName": "{{Microsoft.Storage/storageAccounts}}",
 "primaryAccessKey": "",
 "secondaryAccessKey": "",
 "protectedAccountKeyName": "StorageAccountKey1",
 "blobEndpoint": "https://{{Microsoft.Storage/storageAccounts}}.blob.core.windows.net/",
 "queueEndpoint": "https://{{Microsoft.Storage/storageAccounts}}.queue.core.windows.net/",
 "tableEndpoint": "https://{{Microsoft.Storage/storageAccounts}}.table.core.windows.net/"
 },
 "upgradeMode": "Automatic",
 "availableClusterVersions": [
 {
 "codeVersion": "7.0.466.9590",
 "supportExpiryUtc": "9999-12-31T23:59:59.9999999Z",
 "environment": "Windows"
 }
 ]
 },
 "tags": {},
 "type": "Microsoft.ServiceFabric/clusters",
 "location": "{{location}}"
 }
 ],
 "$schema": "http://schema.management.azure.com/schemas/2015-01-01/deploymentTemplate.json"
}

In [30]:
New-AzResourceGroupDeployment -Name "$resourceGroupName-$((get-date).ToString("yyyyMMdd-hhmmss"))" `
    -ResourceGroupName $resourceGroupName `
    -DeploymentDebugLogLevel All `
    -TemplateFile .\clustertemplate.json `
    -Verbose

VERBOSE: Performing the operation "Creating Deployment" on target "{{resourceGroupName}}".

WARNING: The DeploymentDebug setting has been enabled. This can potentially log secrets like passwords used in resource property or listKeys operations when you retrieve the deployment operations through Get-AzResourceGroupDeploymentOperation

VERBOSE: 14:53:09 - Template is valid.

VERBOSE: 14:53:10 - Create template deployment '{{resourceGroupName}}-20200301-025308'

VERBOSE: 14:53:10 - Checking deployment status in 5 seconds

...

Mode : Incremental

TemplateLink :

Parameters :

Outputs :

DeploymentDebugLogLevel : RequestContent, ResponseContent

In [34]:
# dont use as json. use get-azresource for json
$cluster = get-azservicefabriccluster -Name $clusterName -ResourceGroupName $resourceGroupName
write-host ($cluster)

PSClusterString : AddOnFeatures : 
 AvailableClusterVersions :
 CodeVersion : 7.0.466.9590
 SupportExpiryUtc : 12/31/9999 23:59:59
 Environment : Windows
 AzureActiveDirectory : 
 Certificate :
 Thumbprint : xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
 ThumbprintSecondary : 
 X509StoreName : My
 CertificateCommonNames : 
 ClientCertificateCommonNames :
 ClientCertificateThumbprints :
 ClusterCodeVersion : 7.0.466.9590
 ClusterEndpoint : https://{{location}}.servicefabric.azure.com/runtime/clusters/xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
 ClusterId : xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
 ClusterState : Ready
 DiagnosticsStorageAccountConfig :
 StorageAccountName : {{Microsoft.Storage/storageAccounts}}
 ProtectedAccountKeyName : StorageAccountKey1
 BlobEndpoint : https://{{Microsoft.Storage/storageAccounts}}.blob.core.windows.net/
 QueueEndpoint : https://{{Microsoft.Storage/storageAccounts}}.queue.core.windows.net/
 TableEndpoint : https://{{Microsoft.Storage/storageAccounts}}.table.core.windows.net/
 EventStoreServiceEnabled : 
 FabricSettings :
 Name : Security
 Parameters :
 Name : ClusterProtectionLevel
 Value : EncryptAndSign
 Name : Management
 Parameters :
 Name : CleanupApplicationPackageOnProvisionSuccess
 Value : True
 Name : CleanupUnusedApplicationTypes
 Value : True
 Name : PeriodicCleanupUnusedApplicationTypes
 Value : True
 Name : TriggerAppTypeCleanupOnProvisionSuccess
 Value : True
 Name : MaxUnusedAppTypeVersionsToKeep
 Value : 3
 ManagementEndpoint : https://{{resourceGroupName}}.{{location}}.cloudapp.azure.com:19080
 NodeTypes :
 NodeTypeDescription :
 Name : {{Microsoft.Compute/virtualMachineScaleSets}}
 PlacementProperties : 
 Capacities : 
 ClientConnectionEndpointPort : 19000
 HttpGatewayEndpointPort : 19080
 DurabilityLevel : Bronze
 ApplicationPorts :
 StartPort : 20000
 EndPort : 30000
 EphemeralPorts :
 StartPort : 49152
 EndPort : 65534
 IsPrimary : True
 VmInstanceCount : 3
 ReverseProxyEndpointPort : 19081
 ProvisioningState : Succeeded
 ReliabilityLevel : Bronze
 ReverseProxyCertificate : 
 ReverseProxyCertificateCommonNames : 
 UpgradeDescription : 
 UpgradeMode : Automatic
 VmImage : Windows
 Id : /subscriptions/{{subscription}}/resourcegroups/{{resourceGroupName}}/providers/Microsoft.ServiceFabric/clusters/{{resourceGroupName}}
 Name : {{resourceGroupName}}
 Type : Microsoft.ServiceFabric/clusters
 Location : {{location}}
 Tags :
 Etag : 
 
AddOnFeatures : 
AvailableClusterVersions : {Microsoft.Azure.Management.ServiceFabric.Models.ClusterVersionDetails}
AzureActiveDirectory : 
Certificate : Microsoft.Azure.Management.ServiceFabric.Models.CertificateDescription
CertificateCommonNames : 
ClientCertificateCommonNames : {}
ClientCertificateThumbprints : {}
ClusterCodeVersion : 7.0.466.9590
ClusterEndpoint : https://{{location}}.servicefabric.azure.com/runtime/clusters/xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
ClusterId : xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
ClusterState : Ready
DiagnosticsStorageAccountConfig : Microsoft.Azure.Management.ServiceFabric.Models.DiagnosticsStorageAccountConfig
EventStoreServiceEnabled : 
FabricSettings : {Security, Management}
ManagementEndpoint : https://{{resourceGroupName}}.{{location}}.cloudapp.azure.com:19080
NodeTypes : {{{Microsoft.Compute/virtualMachineScaleSets}}}
ProvisioningState : Succeeded
ReliabilityLevel : Bronze
ReverseProxyCertificate : 
ReverseProxyCertificateCommonNames : 
UpgradeDescription : 
UpgradeMode : Automatic
VmImage : Windows
Id : /subscriptions/{{subscription}}/resourcegroups/{{resourceGroupName}}/providers/Microsoft.ServiceFabric/clusters/{{resourceGroupName}}
Name : {{resourceGroupName}}
Type : Microsoft.ServiceFabric/clusters
Location : {{location}}
Tags : {}
Etag :